ver6为ssd_vgg16网络的map计算代码,不包括模型训练

In [1]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.regularizers import l1
from tensorflow.keras.regularizers import l2
import tensorflow.keras.layers as layers
from tensorflow.keras.layers import InputLayer, UnitNormalization
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, DepthwiseConv2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Dense, Flatten, Concatenate, Reshape, MaxPooling2D, Dropout, ZeroPadding2D
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.layers import InputSpec, Layer
import tensorflow.keras.backend as K
from copy import deepcopy


2023-02-27 21:18:31.104110: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


# 1. ssd标准模型

backbone采用vgg16   
模型输入为300*300图像

In [2]:
def vgg16(input_layer, classes, weight_decay=5e-4):
    
    ## block 1
    ## 300,300,3 -> 150,150,64
    x = Conv2D(64, kernel_size=(3,3),  activation='relu',  padding='same', name='conv1_1', kernel_regularizer=l2(weight_decay))(input_layer)
    x = Conv2D(64, kernel_size=(3,3),  activation='relu',  padding='same', name='conv1_2', kernel_regularizer=l2(weight_decay))(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), padding='same', name='maxpool_1')(x)
    
    # Block 2
    # 150,150,64 -> 75,75,128
    x = Conv2D(128, kernel_size=(3,3),  activation='relu',  padding='same', name='conv2_1', kernel_regularizer=l2(weight_decay))(x)
    x = Conv2D(128, kernel_size=(3,3),  activation='relu',  padding='same', name='conv2_2', kernel_regularizer=l2(weight_decay))(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), padding='same', name='maxpool_2')(x)
    
    # Block 3
    # 75,75,128 -> 38,38,256
    x = Conv2D(256, kernel_size=(3,3),  activation='relu',  padding='same',  name='conv3_1', kernel_regularizer=l2(weight_decay))(x)
    x = Conv2D(256, kernel_size=(3,3),  activation='relu',  padding='same', name='conv3_2', kernel_regularizer=l2(weight_decay))(x)
    x = Conv2D(256, kernel_size=(3,3),  activation='relu',  padding='same', name='conv3_3', kernel_regularizer=l2(weight_decay))(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), padding='same', name='maxpool_3')(x)
    
    # Block 4
    # 38,38,256 -> 19,19,512
    x = Conv2D(512, kernel_size=(3,3),  activation='relu',  padding='same', name='conv4_1', kernel_regularizer=l2(weight_decay))(x)
    x = Conv2D(512, kernel_size=(3,3),  activation='relu',  padding='same', name='conv4_2', kernel_regularizer=l2(weight_decay))(x)
    x = Conv2D(512, kernel_size=(3,3),  activation='relu',  padding='same', name='conv4_3', kernel_regularizer=l2(weight_decay))(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), padding='same', name='maxpool_4')(x)
    
    # Block 5
    # 19,19,512 -> 19,19,512
    x = Conv2D(512, kernel_size=(3,3),  activation='relu',  padding='same',  name='conv5_1', kernel_regularizer=l2(weight_decay))(x)
    x = Conv2D(512, kernel_size=(3,3),  activation='relu',  padding='same', name='conv5_2', kernel_regularizer=l2(weight_decay))(x)
    x = Conv2D(512, kernel_size=(3,3),  activation='relu',  padding='same', name='conv5_3', kernel_regularizer=l2(weight_decay))(x)
    x = MaxPooling2D((3, 3), strides=(1, 1), padding='same', name='maxpool_5')(x)
    
    # FC6
    # 19,19,512 -> 19,19,1024
    x = Conv2D(1024, kernel_size=(3,3),  activation='relu',  padding='same',  name='convfc_6', kernel_regularizer=l2(weight_decay))(x)
    
    # FC7
    # 19,19,1024 -> 19,19,1024
    x = Conv2D(1024, kernel_size=(1,1),  activation='relu',  padding='same', name='convfc_7', kernel_regularizer=l2(weight_decay))(x)
    
    # Block 6
    # 19,19,512 -> 10,10,512
    x = Conv2D(256, kernel_size=(1,1),  activation='relu',  padding='same', name='conv6_1', kernel_regularizer=l2(weight_decay))(x)
    x = ZeroPadding2D(padding=((1, 1), (1, 1)), name='zeropad_1')(x)
    x = Conv2D(512, kernel_size=(3,3),  strides=(2, 2), activation='relu',  padding='valid', name='conv6_2', kernel_regularizer=l2(weight_decay))(x)
    
    # Block 7
    # 10,10,512 -> 5,5,256
    x = Conv2D(128, kernel_size=(1,1),  activation='relu',  padding='same', name='conv7_1')(x)
    x = ZeroPadding2D(padding=((1, 1), (1, 1)), name='zeropad_2')(x)
    x = Conv2D(256, kernel_size=(3,3),  strides=(2, 2), activation='relu',  padding='valid', name='conv7_2')(x)
    
    # Block 8
    # 5,5,256 -> 3,3,256
    x = Conv2D(128, kernel_size=(1,1),  activation='relu',  padding='same', name='conv8_1')(x)
    x = Conv2D(256, kernel_size=(3,3),  strides=(1, 1), activation='relu',  padding='valid', name='conv8_2')(x)
    
    # Block 9
    # 3,3,256 -> 1,1,256
    x = Conv2D(128, kernel_size=(1,1),  activation='relu',  padding='same', name='conv9_1')(x)
    y = Conv2D(256, kernel_size=(3,3),  strides=(1, 1), activation='relu',  padding='valid', name='conv9_2')(x)
    
    model = keras.Model(input_layer, y)
    
    return model

test code

x = layers.Input([300,300,3], 21)
model = vgg16(x, 21)
m_layers = model.layers
print(m_layers[23].name)


In [3]:
def ssd_300_v3(input_shape=[300,300,3], num_classes=21):
    
    """
    y1 = mobi_model.layers[69] 19*19
    y2 = mobi_model.layers[81] 10*10
    y3 = mobi_model.layers[87] 5*5
    y4 = mobi_model.layers[93] 3*3
    y5 = mobi_model.layers[99] 2*2
    y6 = mobi_model.layers[105] 1*1
    """

    in_layer = layers.Input(input_shape)
    vgg16_ = vgg16(in_layer, num_classes)
    
    
    ### ssd架构实现  
    #  38*38  512
    

    x = UnitNormalization(axis=3)(vgg16_.layers[13].output)
#    x = vgg16_.layers[13].output
    num_anchor = 4
    num_loc_pred= 4*num_anchor
    loc1 = Conv2D(filters = num_loc_pred, kernel_size = (3,3), strides=(1,1),
                      padding='same', use_bias = True)(x)
    loc1 = Flatten()(loc1)  

    num_cls_pred = num_classes*num_anchor
    cls1 = Conv2D(filters = num_cls_pred, kernel_size = (3,3), strides=(1,1),
                      padding='same', use_bias = True)(x)
    cls1 = Flatten()(cls1)

    # 对fc7层进行处理 
    #。19*19   1024
    num_anchor = 6
    num_loc_pred= 4*num_anchor
    loc2 = Conv2D(filters = num_loc_pred, kernel_size = (3,3), strides=(1,1),
                      padding='same', use_bias = True)(vgg16_.layers[20].output)
    loc2 = Flatten()(loc2)
    
    num_cls_pred = num_classes*num_anchor
    cls2 = Conv2D(filters = num_cls_pred, kernel_size = (3,3), strides=(1,1),
                      padding='same', use_bias = True)(vgg16_.layers[20].output)
    cls2 = Flatten()(cls2)

    ###  对conv6_2进行处理
    #。10*10.  512
    num_anchor = 6
    num_loc_pred= 4*num_anchor
    loc3 = Conv2D(filters = num_loc_pred, kernel_size = (3,3), strides=(1,1),
                      padding='same', use_bias = True)(vgg16_.layers[23].output)
    loc3 = Flatten()(loc3)
    
    num_cls_pred = num_classes*num_anchor
    cls3 = Conv2D(filters = num_cls_pred, kernel_size = (3,3), strides=(1,1),
                      padding='same', use_bias = True)(vgg16_.layers[23].output)
    cls3 = Flatten()(cls3)

    ### 对conv7_2进行处理
    #。5*5.  256
    num_anchor = 6
    num_loc_pred= 4*num_anchor
    loc4 = Conv2D(filters = num_loc_pred, kernel_size = (3,3), strides=(1,1),
                      padding='same', use_bias = True)(vgg16_.layers[26].output)
    loc4 = Flatten()(loc4)
    
    num_cls_pred = num_classes*num_anchor
    cls4 = Conv2D(filters = num_cls_pred, kernel_size = (3,3), strides=(1,1),
                      padding='same', use_bias = True)(vgg16_.layers[26].output)
    cls4 = Flatten()(cls4)

    ### 对conv8_2进行处理
    #。3*3.    256
    num_anchor = 4
    num_loc_pred= 4*num_anchor
    loc5 = Conv2D(filters = num_loc_pred, kernel_size = (3,3), strides=(1,1),
                      padding='same', use_bias = True)(vgg16_.layers[28].output)
    loc5 = Flatten()(loc5)
    
    num_cls_pred = num_classes*num_anchor
    cls5 = Conv2D(filters = num_cls_pred, kernel_size = (3,3), strides=(1,1),
                      padding='same', use_bias = True)(vgg16_.layers[28].output)
    cls5 = Flatten()(cls5)
    
    #### 对conv9_2进行处理
    #。1*1  256
    num_anchor = 4
    num_loc_pred= 4*num_anchor
    loc6 = Conv2D(filters = num_loc_pred, kernel_size = (3,3), strides=(1,1), 
                      padding='same', use_bias = True)(vgg16_.layers[30].output)
    loc6 = Flatten()(loc6)
    
    num_cls_pred = num_classes*num_anchor
    cls6 = Conv2D(filters = num_cls_pred, kernel_size = (3,3), strides=(1,1), 
                      padding='same', use_bias = True)(vgg16_.layers[30].output)
    cls6 = Flatten()(cls6)
    
    ## 多路输出合并
    localisations = Concatenate(axis=1)([loc1, loc2, loc3, loc4,loc5, loc6])
    localisations = Reshape((-1, 4))(localisations)
    
    logits = Concatenate(axis=1)([cls1, cls2, cls3, cls4, cls5, cls6])
    logits = Reshape((-1, num_classes))(logits)   
    logits = Activation('softmax')(logits)
    
    predictions = Concatenate(axis =-1)([localisations, logits])
    
    ssd_model = keras.Model(inputs=in_layer, outputs=predictions)
    
    return ssd_model 
    

# 2. 数据预处理

各部分主要功能：

1. 生成anchor_boxes.
2. 从annotation文件加载图像和标注数据
3. 计算满足匹配条件的先验框与真实框的相对误差值
4. 根据真实框生成所有目标框的修正值(仅iou大于0.5的目标框为有效目标框的候选)
5. 读取annotaion文件，生成训练数据

## 2.1 anchor_boxes

共有
$38*38*4+19*19*6+10*10*6+5*5*6+3*3*4+1*1*4=8732$ 
个anchor_box

In [4]:
class AnchorBox():
    def __init__(self, input_shape, min_size, max_size=None, aspect_ratios=None, flip=True):
        self.input_shape = input_shape

        self.min_size = min_size
        self.max_size = max_size

        self.aspect_ratios = []
        for ar in aspect_ratios:
            self.aspect_ratios.append(ar)
            self.aspect_ratios.append(1.0 / ar)

    def call(self, layer_shape, mask=None):
        # --------------------------------- #
        #   获取输入进来的特征层的宽和高
        #   比如38x38
        # --------------------------------- #
        layer_height    = layer_shape[0]
        layer_width     = layer_shape[1]
        # --------------------------------- #
        #   获取输入进来的图片的宽和高
        #   比如300x300
        # --------------------------------- #
        img_height  = self.input_shape[0]
        img_width   = self.input_shape[1]

        box_widths  = []
        box_heights = []
        # --------------------------------- #
        #   self.aspect_ratios一般有两个值
        #   [1, 1, 2, 1/2]
        #   [1, 1, 2, 1/2, 3, 1/3]
        # --------------------------------- #
        for ar in self.aspect_ratios:
            # 首先添加一个较小的正方形
            if ar == 1 and len(box_widths) == 0:
                box_widths.append(self.min_size)
                box_heights.append(self.min_size)
            # 然后添加一个较大的正方形
            elif ar == 1 and len(box_widths) > 0:
                box_widths.append(np.sqrt(self.min_size * self.max_size))
                box_heights.append(np.sqrt(self.min_size * self.max_size))
            # 然后添加长方形
            elif ar != 1:
                box_widths.append(self.min_size * np.sqrt(ar))
                box_heights.append(self.min_size / np.sqrt(ar))

        # --------------------------------- #
        #   获得所有先验框的宽高1/2
        # --------------------------------- #
        box_widths  = 0.5 * np.array(box_widths)
        box_heights = 0.5 * np.array(box_heights)

        # --------------------------------- #
        #   每一个特征层对应的步长
        # --------------------------------- #
        step_x = img_width / layer_width
        step_y = img_height / layer_height

        # --------------------------------- #
        #   生成网格中心
        # --------------------------------- #
        linx = np.linspace(0.5 * step_x, img_width - 0.5 * step_x,
                           layer_width)
        liny = np.linspace(0.5 * step_y, img_height - 0.5 * step_y,
                           layer_height)
        centers_x, centers_y = np.meshgrid(linx, liny)
        centers_x = centers_x.reshape(-1, 1)
        centers_y = centers_y.reshape(-1, 1)

        # 每一个先验框需要两个(centers_x, centers_y)，前一个用来计算左上角，后一个计算右下角
        num_anchors_ = len(self.aspect_ratios)
        anchor_boxes = np.concatenate((centers_x, centers_y), axis=1)
        anchor_boxes = np.tile(anchor_boxes, (1, 2 * num_anchors_))
        
        # 获得先验框的左上角和右下角
        anchor_boxes[:, ::4]    -= box_widths
        anchor_boxes[:, 1::4]   -= box_heights
        anchor_boxes[:, 2::4]   += box_widths
        anchor_boxes[:, 3::4]   += box_heights

        # --------------------------------- #
        #   将先验框变成小数的形式
        #   归一化
        # --------------------------------- #
        anchor_boxes[:, ::2]    /= img_width
        anchor_boxes[:, 1::2]   /= img_height
        anchor_boxes = anchor_boxes.reshape(-1, 4)

        anchor_boxes = np.minimum(np.maximum(anchor_boxes, 0.0), 1.0)
        return anchor_boxes

In [5]:
#---------------------------------------------------#
#   用于ssd_mobilenet_V1计算共享特征层的大小
#---------------------------------------------------#
def get_img_output_length(height, width):
    filter_sizes    = [3, 3, 3, 3, 3, 3, 3, 3]
    padding         = [1, 1, 1, 1, 1, 1, 0, 0]
    stride          = [2, 2, 2, 2, 2, 2, 1, 1]
    feature_heights = []
    feature_widths  = []

    for i in range(len(filter_sizes)):
        height  = (height + 2*padding[i] - filter_sizes[i]) // stride[i] + 1
        width   = (width + 2*padding[i] - filter_sizes[i]) // stride[i] + 1
        feature_heights.append(height)
        feature_widths.append(width)
    return np.array(feature_heights)[-6:], np.array(feature_widths)[-6:]

In [6]:
def get_anchors(input_shape = [300,300], anchors_size = [30, 60, 111, 162, 213, 264, 315]):
    feature_heights, feature_widths = get_img_output_length(input_shape[0], input_shape[1])
    aspect_ratios = [[1, 2], [1, 2, 3], [1, 2, 3], [1, 2, 3], [1, 2], [1, 2]]
    anchors = []
    for i in range(len(feature_heights)):
        anchors.append(AnchorBox(input_shape, anchors_size[i], max_size = anchors_size[i+1], 
                    aspect_ratios = aspect_ratios[i]).call([feature_heights[i], feature_widths[i]]))

    anchors = np.concatenate(anchors, axis=0)

    return anchors


# 2. 模型预测

In [7]:
from tensorflow.keras.applications.vgg16 import preprocess_input
from PIL import Image
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from PIL import ImageDraw, ImageFont
import os

## 2.0 用到的函数定义

In [8]:
#---------------------------------------------------#
#   获得分类
#---------------------------------------------------#
def get_classes(classes_path):
    with open(classes_path, encoding='utf-8') as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    return class_names, len(class_names)

In [9]:
#---------------------------------------------------#
#   对输入图像进行resize
#---------------------------------------------------#
def resize_image(image, size, letterbox_image=False):
    iw, ih  = image.size
    w, h    = size
    if letterbox_image:
        scale   = min(w/iw, h/ih)
        nw      = int(iw*scale)
        nh      = int(ih*scale)

        image   = image.resize((nw,nh), Image.BICUBIC)
        new_image = Image.new('RGB', size, (128,128,128))
        new_image.paste(image, ((w-nw)//2, (h-nh)//2))
    else:
        new_image = image.resize((w, h), Image.BICUBIC)
    return new_image

In [10]:
class BBoxUtility(object):
    def __init__(self, num_classes, nms_thresh=0.45, top_k=30):
        self.num_classes    = num_classes
        self._nms_thresh    = nms_thresh
        self._top_k         = top_k
        
    def ssd_correct_boxes(self, box_xy, box_wh, image_shape, letterbox_image=False):
        #-----------------------------------------------------------------#
        #   把y轴放前面是因为方便预测框和图像的宽高进行相乘
        #-----------------------------------------------------------------#
        box_yx = box_xy[..., ::-1]
        box_hw = box_wh[..., ::-1]

        image_shape = np.array(image_shape)

        box_mins    = box_yx - (box_hw / 2.)
        box_maxes   = box_yx + (box_hw / 2.)
        boxes  = np.concatenate([box_mins[..., 0:1], box_mins[..., 1:2], box_maxes[..., 0:1], box_maxes[..., 1:2]], axis=-1)
        boxes *= np.concatenate([image_shape, image_shape], axis=-1)
        return boxes

    def decode_boxes(self, mbox_loc, anchors, variances):
        # 获得先验框的宽与高
        anchor_width     = anchors[:, 2] - anchors[:, 0]
        anchor_height    = anchors[:, 3] - anchors[:, 1]
        # 获得先验框的中心点
        anchor_center_x  = 0.5 * (anchors[:, 2] + anchors[:, 0])
        anchor_center_y  = 0.5 * (anchors[:, 3] + anchors[:, 1])

        # 真实框距离先验框中心的xy轴偏移情况
        decode_bbox_center_x = mbox_loc[:, 0] * anchor_width * variances[0]
        decode_bbox_center_x += anchor_center_x
        decode_bbox_center_y = mbox_loc[:, 1] * anchor_height * variances[1]
        decode_bbox_center_y += anchor_center_y
        
        # 真实框的宽与高的求取
        decode_bbox_width   = np.exp(mbox_loc[:, 2] * variances[2])
        decode_bbox_width   *= anchor_width
        decode_bbox_height  = np.exp(mbox_loc[:, 3] * variances[3])
        decode_bbox_height  *= anchor_height

        # 获取真实框的左上角与右下角
        decode_bbox_xmin = decode_bbox_center_x - 0.5 * decode_bbox_width
        decode_bbox_ymin = decode_bbox_center_y - 0.5 * decode_bbox_height
        decode_bbox_xmax = decode_bbox_center_x + 0.5 * decode_bbox_width
        decode_bbox_ymax = decode_bbox_center_y + 0.5 * decode_bbox_height

        # 真实框的左上角与右下角进行堆叠
        decode_bbox = np.concatenate((decode_bbox_xmin[:, None],
                                      decode_bbox_ymin[:, None],
                                      decode_bbox_xmax[:, None],
                                      decode_bbox_ymax[:, None]), axis=-1)
        # 防止超出0与1
        decode_bbox = np.minimum(np.maximum(decode_bbox, 0.0), 1.0)
        return decode_bbox

    def decode_box(self, predictions, anchors, image_shape, letterbox_image=False, variances = [0.1, 0.1, 0.2, 0.2], confidence=0.5):
        #---------------------------------------------------#
        #   :4是回归预测结果
        #---------------------------------------------------#
        mbox_loc        = predictions[:, :, :4]
        #---------------------------------------------------#
        #   获得种类的置信度
        #---------------------------------------------------#
        mbox_conf       = predictions[:, :, 4:]

        results = []
        #----------------------------------------------------------------------------------------------------------------#
        #   对每一张图片进行处理，由于在predict.py的时候，我们只输入一张图片，所以for i in range(len(mbox_loc))只进行一次
        #----------------------------------------------------------------------------------------------------------------#
        for i in range(len(mbox_loc)):
            results.append([])
            #--------------------------------#
            #   利用回归结果对先验框进行解码
            #--------------------------------#
            decode_bbox = self.decode_boxes(mbox_loc[i], anchors, variances)

            for c in range(1, self.num_classes):
                #--------------------------------#
                #   取出属于该类的所有框的置信度
                #   判断是否大于门限
                #--------------------------------#
                c_confs     = mbox_conf[i, :, c]
                c_confs_m   = c_confs > confidence
                if len(c_confs[c_confs_m]) > 0:
                    #-----------------------------------------#
                    #   取出得分高于confidence的框
                    #-----------------------------------------#
                    boxes_to_process = decode_bbox[c_confs_m]
                    confs_to_process = c_confs[c_confs_m]
                    #-----------------------------------------#
                    #   进行iou的非极大抑制
                    #-----------------------------------------#
                    idx = tf.image.non_max_suppression(tf.cast(boxes_to_process,tf.float32), tf.cast(confs_to_process,tf.float32),
                                            self._top_k,
                                            iou_threshold=self._nms_thresh).numpy()
                    #-----------------------------------------#
                    #   取出在非极大抑制中效果较好的内容
                    #-----------------------------------------#
                    good_boxes  = boxes_to_process[idx]
                    confs       = confs_to_process[idx][:, None]
                    labels      = (c - 1) * np.ones((len(idx), 1))
                    #-----------------------------------------#
                    #   将label、置信度、框的位置进行堆叠。
                    #-----------------------------------------#
                    c_pred      = np.concatenate((good_boxes, labels, confs), axis=1)
                    # 添加进result里
                    results[-1].extend(c_pred)

            if len(results[-1]) > 0:
                results[-1] = np.array(results[-1])
                box_xy, box_wh = (results[-1][:, 0:2] + results[-1][:, 2:4])/2, results[-1][:, 2:4] - results[-1][:, 0:2]
                results[-1][:, :4] = self.ssd_correct_boxes(box_xy, box_wh, image_shape)

        return results

In [11]:
def cvtColor(image):
    if len(np.shape(image))==3 and np.shape(image)[2]==3:
        return image
    else:
        image = image.convert('RGB')
        return image

## 2.2 参数设置和模型加载

In [40]:
class_path = 'model_data/voc_classes.txt'
#---------------------------------------------------------------------#
#   用于预测的图像大小，和train时使用同一个即可
#---------------------------------------------------------------------#
input_shape = [300, 300]
#---------------------------------------------------------------------#
#   只有得分大于置信度的预测框会被保留下来
#---------------------------------------------------------------------#
confidence = 0.5
#---------------------------------------------------------------------#
#   非极大抑制所用到的nms_iou大小
#---------------------------------------------------------------------#
nms_iou = 0.45
#---------------------------------------------------------------------#
#   用于指定先验框的大小
#---------------------------------------------------------------------#
anchors_size  =  [30, 60, 111, 162, 213, 264, 315]
#---------------------------------------------------------------------#
#   该变量用于控制是否使用letterbox_image对输入图像进行不失真的resize，
#   在多次测试后，发现关闭letterbox_image直接resize的效果更好
#---------------------------------------------------------------------#
#letterbox_image =  False

#---------------------------------------------------------------------#
#    保存整个测试集的路径
#---------------------------------------------------------------------#
test_annotation_path = '2007_test.txt'
prediction_path = 'pred_result/'

saved_model = "model_data/voc_trained_weights_1207_0227_5.h5"
classes_path    = 'model_data/voc_classes.txt'

In [41]:
### 1. 获得分类名称和数量
class_names, num_classes = get_classes(classes_path)
anchors = get_anchors(input_shape, anchors_size)

### 2. 生成预测结果集合
preds = {}
for i in range(num_classes):
    preds[class_names[i]]= []

### 3. bbox辅助对象
bbox_util = BBoxUtility(num_classes+1, nms_thresh=nms_iou)

### 3. 读取数据集对应的txt

with open(test_annotation_path, encoding='utf-8') as f:
    test_lines = f.readlines()
num_testfile   = len(test_lines)

### 4. 加载模型
ssd_model = ssd_300_v3([input_shape[0], input_shape[1], 3], num_classes+1)
ssd_model.load_weights(saved_model)

### 5. 预测
for i in range(num_testfile):
    #预处理代码需要修改
#    image, box = get_random_data(test_lines[i], input_shape, random=train)
    line = test_lines[i].split()
    image = Image.open(line[0])
    image_shape = np.array(np.shape(image)[0:2])
    image_filename = line[0].split("/")[-1]
    image_idx = image_filename.split(".")[0]
    
    image_data  = resize_image(image, (input_shape[0], input_shape[1]))

    image_data  = preprocess_input(np.expand_dims(np.array(image_data, dtype='float32'), 0))

    m_pred = ssd_model(image_data, training=False).numpy() #小数据集用本方法 后续考虑用predict方法
    results     = bbox_util.decode_box(m_pred, anchors, image_shape, 
                    confidence=confidence)

    if len(results[0])>0:
        top_label   = np.array(results[0][:, 4], dtype = 'int32')
        top_conf    = results[0][:, 5]
        top_boxes   = results[0][:, :4]
        for j, c in list(enumerate(top_label)):
            res = []
            predicted_class = class_names[int(c)]
            res.append(image_idx)
            res.append(top_conf[j])
            res.append(top_boxes[j])
            preds[predicted_class].append(res)
    if i%100==0:
        print("The {:d} figure have been predicted!", i)

# 6. 保存预测结果
if not os.path.isdir(prediction_path):
    os.mkdir(prediction_path)

for i in range(num_classes):
    pred_file = prediction_path+class_names[i]+'.txt' 
    with open(pred_file, 'w') as f: 
        for j in range(len(preds[class_names[i]])):
            data = preds[class_names[i]][j]
            top, left, bottom, right = data[2]
            top     = max(0, np.floor(top).astype('int32'))
            left    = max(0, np.floor(left).astype('int32'))
            bottom  = min(image.size[1], np.floor(bottom).astype('int32'))
            right   = min(image.size[0], np.floor(right).astype('int32'))
            
            
            # 标注数据的格式应当按照 xmin ，ymin， xmax， ymax来写，而不是行，列的 顺序
            label = '{} {:.2f} {} {} {} {}'.format(data[0], data[1], left, top, right, bottom)
            f.write(label+'\n')
        f.close()
    
    

The {:d} figure have been predicted! 0
The {:d} figure have been predicted! 100
The {:d} figure have been predicted! 200
The {:d} figure have been predicted! 300
The {:d} figure have been predicted! 400
The {:d} figure have been predicted! 500
The {:d} figure have been predicted! 600
The {:d} figure have been predicted! 700
The {:d} figure have been predicted! 800
The {:d} figure have been predicted! 900
The {:d} figure have been predicted! 1000
The {:d} figure have been predicted! 1100
The {:d} figure have been predicted! 1200
The {:d} figure have been predicted! 1300
The {:d} figure have been predicted! 1400
The {:d} figure have been predicted! 1500
The {:d} figure have been predicted! 1600
The {:d} figure have been predicted! 1700
The {:d} figure have been predicted! 1800
The {:d} figure have been predicted! 1900
The {:d} figure have been predicted! 2000
The {:d} figure have been predicted! 2100
The {:d} figure have been predicted! 2200
The {:d} figure have been predicted! 2300
The 

# 3. 模型预测结果MAP计算

In [42]:
#import cPickle
import logging
import numpy as np

import xml.etree.ElementTree as ET

In [43]:
### 从标注文件获取真实的标注结果
def parse_rec(filename):
    """Parse a PASCAL VOC xml file."""
    # 返回一个列表，列表中的每个元素是一个字典
    tree = ET.parse(filename)
    objects = []
    for obj in tree.findall('object'):
        obj_struct = {}
        obj_struct['name'] = obj.find('name').text
        obj_struct['pose'] = obj.find('pose').text
        obj_struct['truncated'] = int(obj.find('truncated').text)
        obj_struct['difficult'] = int(obj.find('difficult').text)
        bbox = obj.find('bndbox')
        obj_struct['bbox'] = [int(bbox.find('xmin').text),
                              int(bbox.find('ymin').text),
                              int(bbox.find('xmax').text),
                              int(bbox.find('ymax').text)]
        objects.append(obj_struct)

    return objects

In [44]:
### 

def voc_ap(rec, prec, use_07_metric=False):
    """Compute VOC AP given precision and recall. If use_07_metric is true, uses
    the VOC 07 11-point method (default:False).
    """
    if use_07_metric:
        # 11 point metric
        ap = 0.
        for t in np.arange(0., 1.1, 0.1):
            if np.sum(rec >= t) == 0:
                p = 0
            else:
                p = np.max(prec[rec >= t])
            ap = ap + p / 11.
    else:
        # correct AP calculation
        # first append sentinel values at the end
        mrec = np.concatenate(([0.], rec, [1.]))
        mpre = np.concatenate(([0.], prec, [0.]))

        # compute the precision envelope
        for i in range(mpre.size - 1, 0, -1):
            mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

        # to calculate area under PR curve, look for points
        # where X axis (recall) changes value
        i = np.where(mrec[1:] != mrec[:-1])[0] #i位数组

        # and sum (\Delta recall) * prec
        ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])  #ap为数组
    return ap

In [45]:
def voc_eval(detpath,
             annopath,
             imagesetfile,
             classname,
             ovthresh=0.5,
             use_07_metric=False):
    """rec, prec, ap = voc_eval(detpath,
                                annopath,
                                imagesetfile,
                                classname,
                                [ovthresh],
                                [use_07_metric])
    Top level function that does the PASCAL VOC evaluation.
    detpath: Path to detections  #检测结果路径
        detpath.format(classname) should produce the detection results file.
    annopath: Path to annotations  #标注文件路径
        annopath.format(imagename) should be the xml annotations file.
    imagesetfile: Text file containing the list of images, one image per line.  #图像集文件名列表文件
    classname: Category name (duh)    # 分类名称
    [ovthresh]: Overlap threshold (default = 0.5)
    [use_07_metric]: Whether to use VOC07's 11 point AP computation
        (default False)
    """
    # assumes detections are in detpath.format(classname)
    # assumes annotations are in annopath.format(imagename)
    # assumes imagesetfile is a text file with each line an image name
    # cachedir caches the annotations in a pickle file

    imageset = os.path.splitext(os.path.basename(imagesetfile))[0]  #获得图像集说明文件的名称
    with open(imagesetfile, 'r') as f:
        lines = f.readlines()
    imagenames = [x.strip() for x in lines]   ##获得图像集文件名列表

    recs = {}
    for i, imagename in enumerate(imagenames):
        recs[imagename] = parse_rec(annopath.format(imagename))
        if i % 100 == 0:
            logger.info(
                'Reading annotation for {:d}/{:d}'.format(
                    i + 1, len(imagenames)))

    # extract gt objects for this class
    class_recs = {}
    npos = 0
    for imagename in imagenames:
        R = [obj for obj in recs[imagename] if obj['name'] == classname]
        bbox = np.array([x['bbox'] for x in R])
        difficult = np.array([x['difficult'] for x in R]).astype(np.bool_)
        det = [False] * len(R)
        npos = npos + sum(~difficult)
        class_recs[imagename] = {'bbox': bbox,
                                 'difficult': difficult,
                                 'det': det}

    # read dets
    detfile = detpath.format(classname)  #一个检测结果为文件中的一行，一张图有多个检测结果就有多行
    with open(detfile, 'r') as f:
        lines = f.readlines()

    splitlines = [x.strip().split(' ') for x in lines]
    image_ids = [x[0] for x in splitlines]

    confidence = np.array([float(x[1]) for x in splitlines])
    BB = np.array([[float(z) for z in x[2:]] for x in splitlines])

    # sort by confidence 按照降序排列，高置信度在前
    sorted_ind = np.argsort(-confidence)
    BB = BB[sorted_ind, :]
    image_ids = [image_ids[x] for x in sorted_ind]  #预测出xx类目标的图像列表

    # go down dets and mark TPs and FPs
    nd = len(image_ids)
    tp = np.zeros(nd)   # true predict
    fp = np.zeros(nd)   # false predict
    for d in range(nd):
        R = class_recs[image_ids[d]]   #xx图对应的gt标注数据
        bb = BB[d, :].astype(float)    #xx图的预测结果坐标
        ovmax = -np.inf
        BBGT = R['bbox'].astype(float)

        if BBGT.size > 0:               #预测框与该图所有的gt框做重叠计算，找到重合度最大的框
            # compute overlaps
            # intersection
            ixmin = np.maximum(BBGT[:, 0], bb[0])
            iymin = np.maximum(BBGT[:, 1], bb[1])
            ixmax = np.minimum(BBGT[:, 2], bb[2])
            iymax = np.minimum(BBGT[:, 3], bb[3])
            iw = np.maximum(ixmax - ixmin + 1., 0.)
            ih = np.maximum(iymax - iymin + 1., 0.)
            inters = iw * ih

            # union
            uni = ((bb[2] - bb[0] + 1.) * (bb[3] - bb[1] + 1.) +
                   (BBGT[:, 2] - BBGT[:, 0] + 1.) *
                   (BBGT[:, 3] - BBGT[:, 1] + 1.) - inters)

            overlaps = inters / uni
            ovmax = np.max(overlaps)
            jmax = np.argmax(overlaps)

        if ovmax > ovthresh:
            if not R['difficult'][jmax]:
                if not R['det'][jmax]:
                    tp[d] = 1.
                    R['det'][jmax] = 1
                else:
                    fp[d] = 1.
        else:
            fp[d] = 1.
    # compute precision recall
    fp = np.cumsum(fp)    #维度相加，得到错误检测结果累计数组
    tp = np.cumsum(tp)    #维度相加，得到正确检测结果累计数组
    rec = tp / float(npos)  #正确目标检测完成率
    # avoid divide by zero in case the first detection matches a difficult
    # ground truth
    prec = tp / np.maximum(tp + fp, np.finfo(np.float64).eps)  #检测结果正确率
    ap = voc_ap(rec, prec, use_07_metric)

    return rec, prec, ap

In [46]:

logger = logging.getLogger(__name__)
detpath = 'pred_result/{:s}.txt'
annopath = '/home/whd/work/deeplearn/data/VOC_data/VOC_2007/VOC2007_test/Annotations/{:s}.xml'
imagesetfile='test.txt'
c_recs=[]
c_precs = []
c_aps = []
for i in range(num_classes):
    c_rec, c_prec, c_ap = voc_eval(detpath, annopath, imagesetfile, class_names[i])
    print("c_rec", c_rec[-1])
    print("c_prec", c_prec[-1])
    print("c_ap", c_ap)
    c_recs.append(c_rec)
    c_precs.append(c_prec)
    c_aps.append(c_ap)

#rec_ave = np.mean(np.array(c_recs))
#prec_ave = np.mean(np.array(c_precs))
ap_ave = np.mean(np.array(c_aps))
#print("The average recall rate of ssg_vgg16 for VOC dataset is {:d}.", rec_ave)
#print("The average precision rate of ssg_vgg16 for VOC dataset is {:d}.", prec_ave)
print("The average ap of ssg_vgg16 for VOC dataset is {:d}.", ap_ave)



c_rec 0.5964912280701754
c_prec 0.8673469387755102
c_ap 0.5853273308764004
c_rec 0.7091988130563798
c_prec 0.8047138047138047
c_ap 0.6430750033036963
c_rec 0.5838779956427015
c_prec 0.8195718654434251
c_ap 0.5403300764243442
c_rec 0.4790874524714829
c_prec 0.72
c_ap 0.4254839796771143
c_rec 0.3070362473347548
c_prec 0.6697674418604651
c_ap 0.2524656600713079
c_rec 0.7089201877934272
c_prec 0.798941798941799
c_ap 0.6500273343904899
c_rec 0.6319733555370525
c_prec 0.7689969604863222
c_ap 0.5573528896904844
c_rec 0.8184357541899442
c_prec 0.8207282913165266
c_ap 0.7557350670878864
c_rec 0.41005291005291006
c_prec 0.5719557195571956
c_ap 0.2962626069167901
c_rec 0.5245901639344263
c_prec 0.757396449704142
c_ap 0.4516295585418265
c_rec 0.7233009708737864
c_prec 0.6898148148148148
c_ap 0.666452185821278
c_rec 0.7934560327198364
c_prec 0.7054545454545454
c_ap 0.7109634579833632
c_rec 0.8017241379310345
c_prec 0.8063583815028902
c_ap 0.7680888258495153
c_rec 0.6615384615384615
c_prec 0.8365758